In [89]:
import geopandas as gpd
import psycopg2
import yaml
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, make_scorer
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [90]:
def read_yaml(file_path):
    """
    read in config file.  All options and adjustments should be made to config file.
    input: yaml file path
    output: dictionary
    """
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

In [91]:
conn_creds = read_yaml('C:\\DSSG_CHAR\\Configs\\db_conn.yaml')

connection = psycopg2.connect(
    dbname=conn_creds['dbname'],
    user=conn_creds['user'],
    password=conn_creds['password'],
    host=conn_creds['address']
)

char_query = "SELECT * FROM char_alerts;"
char_alerts = gpd.GeoDataFrame.from_postgis(char_query, connection, geom_col='centroid')
char_alerts = char_alerts.set_crs(4326)
char_alerts = char_alerts.to_crs(5880)

c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [92]:
biomas_alerts = gpd.read_file(r"c:\DSSG_CHAR\Intermediate_Data_Sets\Mapbiomas_alerts\dashboard_alerts-shapefile.shp")
biomas_alerts = biomas_alerts.to_crs(5880)

c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: c:\DSSG_CHAR\Intermediate_Data_Sets\Mapbiomas_alerts\dashboard_alerts-shapefile.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [93]:
'''
This function calculates the shortest distance from a char_alert to a given feature. This function takes a query, connection credentials, 
the dataframe of char_alerts, the name of the geometry column, and the distance name variable. 
The CRS is set to a projection which is in metres and then performs a spatial join.
'''

def queryToDataFrame(query, connection, char_alerts, geometry_name, distance_name):
    features = gpd.GeoDataFrame.from_postgis(query, connection, geom_col= geometry_name)
    features = features.set_crs(4326)
    features = features.to_crs(5880)
    result = gpd.sjoin_nearest(char_alerts, features, how='left', distance_col= distance_name)
    return result

In [94]:
'''
This function merges two dataframes on their geo_id and drops duplicate values. This is necessary as a consequence of the 
sjoin_nearest function providing multiple outputs for a single input when features are equidistant.
'''

def mergeDataFrame(dfA, dfB, distA, distB):
    dfA = dfA[['geo_id', distA]]
    dfB = dfB[['geo_id', distB]]
    df = dfA.merge(dfB, on='geo_id')
    df = df.drop_duplicates(subset='geo_id')
    return df

In [95]:
all_distances = pd.DataFrame()

In [96]:

'''
This code retrieves features from the database and them merges them two at a time.
'''
rural_roads = queryToDataFrame("SELECT * FROM rural_census_roads;", connection, char_alerts, 'geometry', 'Distance to nearest rural road')
watermass = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'watermass';", connection, char_alerts, 'wkb_geometry','Distance to nearest watermass')
all_distances = mergeDataFrame(rural_roads, watermass, 'Distance to nearest rural road', 'Distance to nearest watermass')

towns = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'towns';", connection, char_alerts, 'wkb_geometry', 'Distance to nearest town')
vilas = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'vilas';", connection, char_alerts, 'wkb_geometry', 'Distance to nearest vila')
all_distances = all_distances.merge(mergeDataFrame(towns, vilas, 'Distance to nearest town', 'Distance to nearest vila'), on='geo_id')

rural_settlements = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'rural_settlements';", connection, char_alerts, 'wkb_geometry', 'Distance to nearest rural settlements')
mineral_extraction = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'mineral_extraction_points';", connection, char_alerts, 'wkb_geometry', 'Distance to nearest mineral extraction point')
all_distances = all_distances.merge(mergeDataFrame(rural_settlements, mineral_extraction, 'Distance to nearest rural settlements', 'Distance to nearest mineral extraction point'), on='geo_id')

landing_path = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'landing_path';", connection, char_alerts, 'wkb_geometry', 'Distance to nearest landing path')
islands = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'islands';", connection, char_alerts, 'wkb_geometry', 'Distance to nearest island')
all_distances = all_distances.merge(mergeDataFrame(landing_path, islands, 'Distance to nearest landing path', 'Distance to nearest island'), on='geo_id')

dense_infrastructure = queryToDataFrame("SELECT * FROM geospatial_view WHERE feature_type = 'densely_infrastructured_areas';", connection, char_alerts, 'wkb_geometry', 'Distance to nearest densely infrastructured area')
ibama_embargo = queryToDataFrame("SELECT * FROM ibama_embargo", connection, char_alerts, 'wkb_geometry', 'Distance to nearest ibama embargo')
all_distances = all_distances.merge(mergeDataFrame(dense_infrastructure, ibama_embargo, 'Distance to nearest densely infrastructured area', 'Distance to nearest ibama embargo'), on='geo_id')

indigenous_lands = queryToDataFrame("SELECT * FROM indigenous_lands", connection, char_alerts, 'wkb_geometry', 'Distance to nearest indigenous lands')
official_roads = queryToDataFrame("SELECT * FROM official_roads", connection, char_alerts, 'wkb_geometry', 'Distance to nearest official roads')
all_distances = all_distances.merge(mergeDataFrame(indigenous_lands, official_roads, 'Distance to nearest indigenous lands', 'Distance to nearest official roads'), on='geo_id')

deforestation_permit = queryToDataFrame("SELECT * FROM deforestation_permits;", connection, char_alerts, 'wkb_geometry', 'Distance to nearest deforestation permit')
mpb_alerts = gpd.sjoin_nearest(char_alerts, biomas_alerts, how='left', distance_col= 'Distance to nearest biomas alert')
all_distances = all_distances.merge(mergeDataFrame(deforestation_permit, mpb_alerts, 'Distance to nearest deforestation permit', 'Distance to nearest biomas alert'))




c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sq

,geo_id,Char alerts within 10km
0,2ef5af51-ab0b-4fd5-8d47-a29a9482f309,6
1,52446ccf-24ab-4cb9-8e89-9b469d651622,5
2,3230edca-3497-4e2c-9b7e-dd26d6aeeb5b,4
3,150a530e-7f5c-4e3e-8696-129cddf2f0fc,3
4,259150da-3c22-4c1b-aff5-a8b832e8996b,2
...,...,...
5273,760cae5a-bcaf-4a35-8493-0ad1d4f2de21,0
5274,d591475c-4e83-40ee-81c6-6e89527e2ce3,7
5275,e159eeb5-5f1b-44f2-8cb0-1ee3727a8394,10
5276,4068a471-d817-467f-972a-cf1287a169e6,11


In [98]:
''''
Count the number of char alerts within a 10km radius but more than 50m of a char alert. Group by location to avoid counting same site over time.
'''

# creating unique x_tile y_tile combination for spatial grouping
df = char_alerts.copy()
df['loc'] = df['x_tile'].astype(str) + ',' + df['y_tile'].astype(str)
df["validation"] = df["validation"].apply(lambda x: 1 if x else 0)
df_no_duplicates = df.drop_duplicates(subset='loc', keep='first')

# Create a buffer with a 10 km radius
char_buffer_10km = df_no_duplicates.copy()
char_buffer_10km['geometry'] = char_buffer_10km.buffer(10000)
char_buffer_10km = char_buffer_10km.set_geometry('geometry')

# Create a buffer with a 100 m radius
char_buffer_50m = df_no_duplicates.copy()
char_buffer_50m['geometry'] = char_buffer_50m.buffer(100)
char_buffer_50m = char_buffer_50m.set_geometry('geometry')

points_in_10km = gpd.sjoin(df_no_duplicates, char_buffer_10km, how='left', predicate='within')
points_in_50m = gpd.sjoin(df_no_duplicates, char_buffer_50m, how='left', predicate='within')

# Count the number of char alerts within the 10 km buffer
char_alerts_within_10km = points_in_10km.groupby('geo_id_left').size().reset_index(name='Char alerts within 10km')
char_alerts_within_50m = points_in_50m.groupby('geo_id_left').size().reset_index(name='Char alerts within 50m')

char_alerts_within_10km['Char alerts within 10km'] = char_alerts_within_10km['Char alerts within 10km'] - char_alerts_within_50m['Char alerts within 50m']

# Rename columns to match the final output
char_alerts_within_10km = char_alerts_within_10km.rename(columns={'geo_id_left': 'geo_id'})
char_alerts_within_10km = char_alerts_within_10km.merge(df_no_duplicates, on='geo_id')
char_alerts_within_10km = char_alerts_within_10km[['geo_id', 'Char alerts within 10km', 'loc']]
char_alerts_within_10km
df = df.merge(char_alerts_within_10km, on='loc')
df = df[['geo_id_x', 'Char alerts within 10km']].rename(columns={'geo_id_x': 'geo_id'})



''''
Count the number of mapbiomas alerts within a 10km radius of a char alert.
'''
char_buffer = char_alerts.copy()
char_buffer['geometry'] = char_buffer.buffer(10000)
char_buffer = char_buffer.set_geometry('geometry')
points_in_buffer = gpd.sjoin(biomas_alerts, char_buffer, how='right', predicate='within')
MPB_alerts_within_10km = points_in_buffer.groupby('geo_id').size().reset_index(name='MPB alerts within 10km')
MPB_alerts_within_10km


,geo_id,MPB alerts within 10km
0,000dd062-a882-469f-bfd3-d40b5cc648e8,13
1,00187079-074c-4064-a608-48dc50d8b83f,1
2,001cd8f6-979f-4310-b7fc-ee207cd257f8,1
3,0024e373-cdf9-41c0-8d7d-0322eb2a9f24,55
4,00275a2d-7ee1-4a83-8ca1-aa3b36819776,7
...,...,...
5273,ffbf29d4-05e4-4296-9575-0f4fc76f1ef3,55
5274,ffcdbe7d-c334-445c-8543-cd9f15466b15,1
5275,ffd8e48f-a897-47bd-ae6d-d2dd7422ea17,32
5276,ffe8fe56-7bdd-46c2-9234-440d27357833,1


In [99]:
all_distances = all_distances.merge(mergeDataFrame(df, MPB_alerts_within_10km, 'Char alerts within 10km', 'MPB alerts within 10km'))

In [100]:
all_distances

,geo_id,Distance to nearest rural road,Distance to nearest watermass,Distance to nearest town,Distance to nearest vila,Distance to nearest rural settlements,Distance to nearest mineral extraction point,Distance to nearest landing path,Distance to nearest island,Distance to nearest densely infrastructured area,Distance to nearest ibama embargo,Distance to nearest indigenous lands,Distance to nearest official roads,Distance to nearest deforestation permit,Distance to nearest biomas alert,Char alerts within 10km,MPB alerts within 10km
0,2ef5af51-ab0b-4fd5-8d47-a29a9482f309,30.036099,962.017296,8710.060539,42015.594965,17673.364950,62998.760315,73103.171599,1134.956652,7763.770395,3936.177257,40353.180407,3804.465484,111621.956597,4887.593094,6,2
1,52446ccf-24ab-4cb9-8e89-9b469d651622,8.237546,923.384833,8517.950577,42369.746726,17797.048351,62674.984727,72812.642488,1056.347325,7582.814129,4083.330015,40743.796757,3704.039521,111278.215657,4644.837723,5,2
2,3230edca-3497-4e2c-9b7e-dd26d6aeeb5b,1195.465385,10223.194189,12515.503700,59270.904773,3205.961938,23643.227277,36251.358510,10893.365965,11569.015745,8194.290042,89662.839857,3284.860547,52620.084746,4888.429594,4,16
3,150a530e-7f5c-4e3e-8696-129cddf2f0fc,412.500111,54796.600559,23150.767991,100186.664259,11145.247545,68450.201220,50292.873917,55573.182134,21200.853832,5636.085396,101622.475980,10876.703178,41275.416657,0.000000,3,18
4,259150da-3c22-4c1b-aff5-a8b832e8996b,872.720200,54310.751779,25575.977744,100385.291932,13544.922506,67888.309598,47637.355050,55061.190356,23636.052368,4744.755801,103969.747471,11235.951800,38944.686399,625.705588,2,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5273,760cae5a-bcaf-4a35-8493-0ad1d4f2de21,423.973062,0.000000,13747.505016,15252.892520,3475.146915,145168.550763,106069.766316,13112.358256,14349.552563,12101.980869,217876.188913,6384.283548,37923.197325,2210.538582,0,62
5274,d591475c-4e83-40ee-81c6-6e89527e2ce3,2113.493714,4442.607435,14619.029607,25816.958527,2809.011704,158648.659712,101423.492052,16538.180290,2699.399960,8149.262241,214442.088602,2172.380159,21909.155110,299.241390,7,159
5275,e159eeb5-5f1b-44f2-8cb0-1ee3727a8394,293.734065,2938.122932,4627.620554,40531.449208,6023.185764,177542.620656,98214.657870,5113.662375,2882.832428,2312.633485,212379.806232,303.047280,4061.771983,0.000000,10,49
5276,4068a471-d817-467f-972a-cf1287a169e6,814.943705,4305.257787,4267.220110,41296.668256,3867.545963,175703.493703,99326.453180,5689.471641,2873.115082,3506.473404,211388.002249,2057.843841,2277.180181,650.669734,11,46


In [101]:
all_distances.to_parquet(r"C:\DSSG_CHAR\Intermediate_Data_Sets\all_distances.parquet")

In [102]:
# spatial join char_alerts with what municipality it's in.
municipalities = gpd.GeoDataFrame.from_postgis('select * from adm_division where adm_level = 5', connection, geom_col='geometry')
char_municipality = gpd.sjoin_nearest(char_alerts, municipalities, "left")
char_municipality = char_municipality[['geo_id', 'adm_name']]

c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
C:\Users\leonry\AppData\Local\Temp\9\ipykernel_11612\2853729778.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:5880
Right CRS: EPSG:4326

  char_municipality = gpd.sjoin_nearest(char_alerts, municipalities, "left")
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [103]:
# Query smartlab data on whether a municipality in Maranhao is combating slave labour and join to char_alerts
slave_labour = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_estadic_munic where cd_indicador_spai = '91_06_01_02'", connection, geom_col='wkb_geometry')
slave_labour = slave_labour[['nm_municipio', 'spai_vl_indicador']].rename(columns={"nm_municipio": "adm_name","spai_vl_indicador" : "slave_labour"})

# Fix incorrect municipality name
char_municipality['adm_name'] = char_municipality['adm_name'].apply(lambda x: "Olho d'Água das Cunhãs" if x=="Olho dÁgua das Cunhãs" else x)
result = char_municipality.merge(slave_labour, on='adm_name').drop_duplicates(subset='geo_id')

# Query smartlab data on whether a municipality in Maranhao is combating slave labour and join to char_alerts
active_search = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_estadic_munic where cd_indicador_spai = '91_06_02_01'", connection, geom_col='wkb_geometry')
active_search = active_search[['nm_municipio', 'spai_vl_indicador']].rename(columns={"nm_municipio": "adm_name","spai_vl_indicador" : "active_search"})
result = result.merge(active_search, on='adm_name')

# Query smartlab data on Program, Plan, or Action Developed by Municipal Management - Combating Forced or Slave Labor and Reintegration of its Victims
slave_labour_reintegration = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_estadic_munic where cd_indicador_spai = '91_06_01_03'", connection, geom_col='wkb_geometry')
slave_labour_reintegration = slave_labour_reintegration[['nm_municipio', 'spai_vl_indicador']].rename(columns={"nm_municipio": "adm_name","spai_vl_indicador" : "slave_reintegration"})
result = result.merge(slave_labour_reintegration, on='adm_name')

# Query smartlab data on Program for the Fight Against Slave Labor and Reintegration of its Victims
combat_slave_labour = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_estadic_munic where cd_indicador_spai = 'MUNIC_MDHU5710'", connection, geom_col='wkb_geometry')
combat_slave_labour = combat_slave_labour[['nm_municipio', 'spai_vl_indicador']].rename(columns={"nm_municipio": "adm_name","spai_vl_indicador" : "combat_slave_labour"})
result = result.merge(combat_slave_labour, on='adm_name').drop_duplicates(subset='geo_id')

columns_to_convert = ['slave_labour', 'active_search', 'slave_reintegration', 'combat_slave_labour']
result[columns_to_convert] = result[columns_to_convert].astype(int)
# Replace all 4's with 0's
result[columns_to_convert] = result[columns_to_convert].replace(4, 0)

c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pand

In [104]:
# Query smartlab data on literacy rate
literacy_rate = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_indicadores where cd_indicador = '01_11_09_00'", connection, geom_col='wkb_geometry')
literacy_rate = literacy_rate[['nm_municipio', 'vl_indicador']].rename(columns={"nm_municipio": "adm_name","vl_indicador" : "literacy rate"})
result = result.merge(literacy_rate, on='adm_name').drop_duplicates(subset='geo_id')

# Query smartlab data on poverty rate
poverty_rate = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_indicadores where cd_indicador = '01_14_13_00'", connection, geom_col='wkb_geometry')
poverty_rate = poverty_rate[['nm_municipio', 'vl_indicador']].rename(columns={"nm_municipio": "adm_name","vl_indicador" : "poverty rate"})
result = result.merge(poverty_rate, on='adm_name').drop_duplicates(subset='geo_id')

# Query smartlab data on HDI score
HDI_score = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_indicadores where cd_indicador = '06_01_07_11'", connection, geom_col='wkb_geometry')
HDI_score = HDI_score[['nm_municipio', 'vl_indicador']].rename(columns={"nm_municipio": "adm_name","vl_indicador" : "HDI score"})
result = result.merge(HDI_score, on='adm_name').drop_duplicates(subset='geo_id')

# Query smartlab data on rescued slaves
num_rescued = gpd.GeoDataFrame.from_postgis("select * from smartlab_central_indicadores_mun  where  cd_indicador = 'te_sit_trab_resgatados'", connection, geom_col='wkb_geometry')
num_rescued = num_rescued[['nm_municipio', 'vl_indicador']].rename(columns={"nm_municipio": "adm_name","vl_indicador" : "rescued workers"})
result = result.merge(num_rescued, on='adm_name', how='left').drop_duplicates(subset='geo_id')
result['rescued workers'] = result['rescued workers'].fillna(0)

# Query smartlab data on other work alerts
work_alerts = gpd.GeoDataFrame.from_postgis("select * from public.smartlab_indicadores where cd_indicador = '06_05_10_00'", connection, geom_col='wkb_geometry')
work_alerts = work_alerts[['nm_municipio', 'vl_indicador']].rename(columns={"nm_municipio": "adm_name","vl_indicador" : "work alerts"})
result = result.merge(work_alerts, on='adm_name').drop_duplicates(subset='geo_id')


c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
c:\Users\leonry\.conda\envs\dssg_dev\Lib\site-packages\geopandas\io\sql.py:185: UserWarning: pand

In [105]:
result = result.drop(columns=['adm_name'])

In [106]:
result.to_parquet(r"C:\DSSG_CHAR\Intermediate_Data_Sets\smartlab_data.parquet")